# Image Utils

The `img_utils` module alows users to manage images datasets (folders and images files) directly on your disk.

### Type Of Folder Trees

#### tree = 1
```
[root_path] "folder_name"/..  
    [subfolder] class_0/..  
    [subfolder] class_1/..  
    [subfolder] errors/..
```

#### tree = 2
```
[root_path] "folder_name"/..  
    [subfolder] train_tr/..  
        [subfolder] class_0/..  
        [subfolder] class_1/..  
    [subfolder] train_val/..  
        [subfolder] class_0/..  
        [subfolder] class_1/..  
    [subfolder] test/test/..  
    [subfolder] errors/..  
```

### impor `img_utils`

In [21]:
from gapml.utils.img_tools import img_utils
import os

In [5]:
# Proccess root_path = flower_photos
gap = img_utils(root_path='flower_photos')

In [6]:
# current working directory path.
%pwd

'/mnt/c/Users/david/github/gapml/Gap/train'

In [7]:
# current list directory contents
%ls

 10nc.pdf*                  boardgames.csv*    'session 3.ipynb'*
 4scan.pdf*                 crash_2015.pdf*    'session 4.ipynb'*
'NLP Epipog - I.pptx'*      flower_photos/     'session 5.ipynb'*
'Roadmap - Gap 1.0.pptx'*  'session 0.ipynb'*   tut_files/
'Session 6.ipynb'*         'session 1.ipynb'*
'Train - Intro.pptx'*      'session 2.ipynb'*


In [8]:
# root_path labels directory contents
%ls flower_photos/

daisy/  dandelion/  roses/  sunflowers/  tulips/


In [11]:
# transform directoies between tree 1 to tree 2
gap.transform(shufle=True, img_split=0.2)

In [12]:
# notice the new folder flower_photos_t2
%ls

 10nc.pdf*                  boardgames.csv*    'session 3.ipynb'*
 4scan.pdf*                 crash_2015.pdf*    'session 4.ipynb'*
'NLP Epipog - I.pptx'*      flower_photos_t2/  'session 5.ipynb'*
'Roadmap - Gap 1.0.pptx'*  'session 0.ipynb'*   tut_files/
'Session 6.ipynb'*         'session 1.ipynb'*
'Train - Intro.pptx'*      'session 2.ipynb'*


In [13]:
# let's see the structure inside
%ls flower_photos_t2/

errors/  test/  train_tr/  train_val/


In [14]:
# Proccess root_path = flower_photos_t2/train_tr
gap = img_utils(root_path='flower_photos_t2/train_tr', transf='2to1')

In [15]:
# transform directories between tree 2 to tree 1
gap.transform()

In [16]:
# notice we got the previous folder flower_photos
%ls

 10nc.pdf*                  boardgames.csv*    'session 3.ipynb'*
 4scan.pdf*                 crash_2015.pdf*    'session 4.ipynb'*
'NLP Epipog - I.pptx'*      flower_photos/     'session 5.ipynb'*
'Roadmap - Gap 1.0.pptx'*  'session 0.ipynb'*   tut_files/
'Session 6.ipynb'*         'session 1.ipynb'*
'Train - Intro.pptx'*      'session 2.ipynb'*


In [17]:
# let's see the structure inside
%ls flower_photos/

daisy/  dandelion/  roses/  sunflowers/  tulips/


In [57]:
# Proccess root_path = flower_photos
gap = img_utils(root_path='flower_photos')

In [58]:
# Let's extract a sample of the data set
gap.img_container(action='copy', spl=15)

In [20]:
# notice the new folder flower_photos_spl
%ls

 10nc.pdf*                  boardgames.csv*     'session 2.ipynb'*
 4scan.pdf*                 crash_2015.pdf*     'session 3.ipynb'*
'NLP Epipog - I.pptx'*      flower_photos/      'session 4.ipynb'*
'Roadmap - Gap 1.0.pptx'*   flower_photos_spl/  'session 5.ipynb'*
'Session 6.ipynb'*         'session 0.ipynb'*    tut_files/
'Train - Intro.pptx'*      'session 1.ipynb'*


In [22]:
# Let's see inside
%ls flower_photos_spl/

daisy/  dandelion/  roses/  sunflowers/  tulips/


In [23]:
# Proccess root_path = flower_photos
gap = img_utils(root_path='flower_photos', tree=2)

In [24]:
# Let's extract a sample of the data set but with the tree 2 structure
gap.img_container(action='copy', spl=15, shufle=True, img_split=0.2)

In [29]:
img_list_tr = os.listdir('flower_photos_t2_spl/train_tr/roses')
img_list_val = os.listdir('flower_photos_t2_spl/train_val/roses')
print('number of images in train_tr:', len(img_list_tr))
print('number of images in train_val:', len(img_list_val))
%ls flower_photos_t2_spl/train_tr

number of images in train_tr: 13
number of images in train_val: 2
daisy/  dandelion/  roses/  sunflowers/  tulips/


In [30]:
# Proccess root_path = flower_photos_spl
gap = img_utils(root_path='flower_photos_spl')

In [59]:
# current images names
%ls flower_photos_spl/roses/

10090824183_d02c613f10_m.jpg*  118974357_0faa23cce9_n.jpg*
102501987_3cdb8e5394_n.jpg*    11944957684_2cc806276e.jpg*
10503217854_e66a804309.jpg*    12045735155_42547ce4e9_n.jpg*
10894627425_ec76bbc757_n.jpg*  12165480946_c4a3fe182d_n.jpg*
110472418_87b6a3aa98_m.jpg*    12202373204_34fb07205b.jpg*
11102341464_508d558dfc_n.jpg*  12238827553_cf427bfd51_n.jpg*
11233672494_d8bf0a3dbf_n.jpg*  12240165555_98625b1e88_n.jpg*
11694025703_9a906fedc1_n.jpg*


In [38]:
# rename images
# try the diferent options
gap.img_rename(text='test')# text=True text='test'

In [39]:
# new images names in the label roses
%ls flower_photos_spl/roses/

test_0.jpg*   test_12.jpg*  test_16.jpg*  test_2.jpg*  test_6.jpg*
test_1.jpg*   test_13.jpg*  test_17.jpg*  test_3.jpg*  test_7.jpg*
test_10.jpg*  test_14.jpg*  test_18.jpg*  test_4.jpg*  test_8.jpg*
test_11.jpg*  test_15.jpg*  test_19.jpg*  test_5.jpg*  test_9.jpg*


In [40]:
# replace part of the name text 
gap.img_replace(old='st', new='sted', img_id=False)

In [42]:
# new images names in the label roses
%ls flower_photos_spl/roses/

tested_0.jpg*   tested_12.jpg*  tested_16.jpg*  tested_2.jpg*  tested_6.jpg*
tested_1.jpg*   tested_13.jpg*  tested_17.jpg*  tested_3.jpg*  tested_7.jpg*
tested_10.jpg*  tested_14.jpg*  tested_18.jpg*  tested_4.jpg*  tested_8.jpg*
tested_11.jpg*  tested_15.jpg*  tested_19.jpg*  tested_5.jpg*  tested_9.jpg*


In [46]:
# Proccess root_path = flower_photos_t2_spl/train_tr
gap = img_utils(root_path='flower_photos_t2_spl/train_tr', tree=2)

In [47]:
# current images names
%ls flower_photos_t2_spl/train_tr/roses/

102501987_3cdb8e5394_n.jpg*    12202373204_34fb07205b.jpg*
10894627425_ec76bbc757_n.jpg*  12238827553_cf427bfd51_n.jpg*
11102341464_508d558dfc_n.jpg*  12240165555_98625b1e88_n.jpg*
11233672494_d8bf0a3dbf_n.jpg*  12240303_80d87f77a3_n.jpg*
118974357_0faa23cce9_n.jpg*    12240577184_b0de0e53ea_n.jpg*
11944957684_2cc806276e.jpg*    12243068283_ee4c2683e2_n.jpg*
12165480946_c4a3fe182d_n.jpg*


In [50]:
# rename images
# try the diferent options
gap.img_rename()# text=True text='test'

In [51]:
# new images names in the label roses
%ls flower_photos_t2_spl/train_tr/roses/

test_0.jpg*   test_11.jpg*  test_3.jpg*  test_6.jpg*  test_9.jpg*
test_1.jpg*   test_12.jpg*  test_4.jpg*  test_7.jpg*
test_10.jpg*  test_2.jpg*   test_5.jpg*  test_8.jpg*


In [52]:
gap.img_replace(old='st', new='sted', img_id=True)

In [53]:
# new images names in the label roses
%ls flower_photos_t2_spl/train_tr/roses/

tested_0_0.jpg*   tested_1_1.jpg*   tested_5_2.jpg*  tested_9_6.jpg*
tested_10_7.jpg*  tested_2_10.jpg*  tested_6_3.jpg*
tested_11_8.jpg*  tested_3_11.jpg*  tested_7_4.jpg*
tested_12_9.jpg*  tested_4_12.jpg*  tested_8_5.jpg*


### Delete samples

In [60]:
gap = img_utils(root_path='flower_photos_spl', rm=True)

Warning! this will delete your image dataset. Are you sure? [Yes/no]: y
Your files were deleted


In [55]:
gap = img_utils(root_path='flower_photos_t2_spl', rm=True)

Warning! this will delete your image dataset. Are you sure? [Yes/no]: y
Your files were deleted


In [56]:
# current list directory contents 
%ls

 10nc.pdf*                  boardgames.csv*    'session 3.ipynb'*
 4scan.pdf*                 crash_2015.pdf*    'session 4.ipynb'*
'NLP Epipog - I.pptx'*      flower_photos/     'session 5.ipynb'*
'Roadmap - Gap 1.0.pptx'*  'session 0.ipynb'*   tut_files/
'Session 6.ipynb'*         'session 1.ipynb'*
'Train - Intro.pptx'*      'session 2.ipynb'*
